# TP1 du module 7 : l'apprentissage non supervisé

Dans ce TP, nous allons mettre en pratique les principes de l'apprentissage non supervisé. Objectifs :
* Passer en revue les principaux algorithmes de clustering
* Comparer les performances de ces différents algorithmes
* Comparer avec les performances de la classification supervisée.

La recherche d'itemsets fréquents et de règles d'associaion ne sera pas abordée dans ce TP.

Même si ce module concerne l'apprentissage non supervisé, nous allons continuer à explorer le jeu de données du Titanic : cela nous permettra, à la fin, de comparer les clusters obtenus, avec les deux classes réelles obtenus via les labels du jeu de données.

Nous allons donc commencer par charger les données. Pour cela, repartez du csv obtenu à la fin du TP1 du module 4. Construisez, comme pour l'apprentissage supervisé, deux dataframe : un avec les attributs, l'autre avec les labels.

In [69]:
import pandas as pd

titanic = pd.read_csv("Titanic.csv")

X = titanic.drop(['Survived'], axis=1)
y = titanic['Survived']
X

## Découverte de kmeans

1. Nous allons commencer par faire un premier clustering avec KMeans. Comme nous connaissons le nombre de clusters à rechercher, créer un modèle avec la classe de scikit-learn, en fixant le nombre de clusters. Appliquez ce clustering aux attributs de titanic, et récupérer dans une liste le numéro du cluster prédit pour chaque donnée.

In [70]:
from sklearn.cluster import KMeans

# Création du modèle grâce à la classe KMeans de Scikit-learn
kmeans = KMeans(n_clusters=2, n_init='auto')

# Apprentissage des clusters à partir des données
kmeans.fit(X)

# Pour chaque point, récupérer le numéro du cluster auquel il est affecté
y_kmeans = kmeans.predict(X)

y_kmeans

2. Quelle est la classe majoritaire dans le premier cluster ? Dans le deuxième cluster ? Affichez une matrice, qui donne pour chaque classe le nombre de fois où elle apparait dans chaque cluster. Qu'observez-vous ?

In [71]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y, y_kmeans)
cm

3. Faite un pairplot des données du Titanic, en colorant chaque donnée en fonction du cluster auquel elle a été affectée. Remarquez-vous des phénomènes intéressants ?

In [72]:
X['y_kmeans'] = y_kmeans
sns.pairplot(X, hue='y_kmeans')

4. Les algorithmes de clustering impliquant des mesures de distances, ils sont très sensibles aux plages de valeurs des différents attributs. Normalisez vos données, et refaites les mêmes étapes que précédemment. Observez-vous une différence ?

In [77]:
from sklearn.preprocessing import normalize

X_normalized= normalize(X)

kmeans = KMeans(n_clusters=2, n_init='auto')
y_kmeans_normalize = kmeans.fit_predict(X_normalized)
cm = confusion_matrix(y, y_kmeans_normalize)
cm

In [78]:
X_normalize_df= pd.DataFrame(X_normalized)
X_normalize_df['y_kmeans_normalize'] = y_kmeans_normalize
sns.pairplot(X_normalize_df, hue='y_kmeans_normalize')

5. Etudions l'évolution du coefficient de silhouette. Pour k variant de 2 à 10, tracez l'évolution de ce coefficient pour un clustering kmeans appliqué au données normalisées. Le nombre de clusters utilisé au question précédentes vous semble-t'il pertinent ?

In [80]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score

k_values = np.arange(2, 11)

silhouette_scores = []

for k in k_values:
    model = KMeans(n_clusters=k, n_init='auto')
    y_results = model.fit_predict(X_normalized)
    silhouette = silhouette_score(X_normalized, y_results)
    silhouette_scores.append(silhouette)

print("Le meilleur coefficient ", np.max(silhouette_scores), " est pour k =", np.argmax(silhouette_scores) + 2)

plt.plot(k_values, silhouette_scores)
plt.xticks(k_values)
plt.xlabel("Nombre clusters (k)")
plt.ylabel("Score silhouette")
plt.title("Evolution de score de silhouette en fonction du nombre de clusters")
plt.show()

## Clustering hiérarchique

Pour toute la suite du TP, nous travaillerons avec les données normalisées. 

1. Créez un modèle de clustering hiérarchique (`AgglomerativeClustering`). En utilisant le coefficient de silhouette, comparez le score obtenu pour deux clusters avec celui obtenu par KMeans sur la section précédente.

In [82]:
from sklearn.cluster import AgglomerativeClustering

hierarchical = AgglomerativeClustering(n_clusters=2)
y_kmeans = hierarchical.fit_predict(X_normalized)
print(silhouette_score(X_normalized, y_kmeans))

2. Représentez graphiquement ce nouveau clustering, à l'aide d'un pairplot. Remarquez-vous des tendances ou des changements intéressants ?

In [84]:
X_normalize_df= pd.DataFrame(X_normalized)
X_normalize_df['y_kmeans_normalize'] = y_kmeans_normalize
sns.pairplot(X_normalize_df, hue='y_kmeans_normalize')

3. Etudiez l'impact du paramètre `linkage` sur les résultats de votre clustering hiérarchique. Pour rappel, ce paramètre désigne la manière dont est calculée la distance entre deux clusters, pour décider lesquels réunir à une itération donnée. construisez un graphique montrant la valeur du coefficient silhouette en fonction de la méthode utilisée.

In [85]:
methods= ('ward', 'complete', 'average', 'single')
silhouettes = []

for m in methods:
    hierarchical = AgglomerativeClustering(n_clusters=2, linkage=m)
    y_ac = hierarchical.fit_predict(X_normalized)
    silhouettes.append(silhouette_score(X_normalized, y_ac))
    
fig= plt.subplots(figsize=(15,10))
plt.plot(methods, silhouettes)
plt.title("Evolution du coef ")
plt.xlabel('Methode')
plt.ylabel('Coef de silhouette')
plt.show()

## Comparaison des différents clustering

Nous allons à présent comparer les résultats obtenus avec les autres algorithmes de clustering proposés par Scikit-learn. Testez ces différents algorithmes, et calculez à chaque fois le coefficient de slihouette obtenu. Pour les algorithmes qui ne demandent pas de préciser le nombre de clusters à construire, affichez le nombre de clusters déduit par l'algorithme.

Présentez une synthèse de vos résultats sous forme d'un tableau et d'un graphique.

Liste des algorithmes à prendre en compte : KMeans, DBScan, spectral, affinity_propagation, agglomerativeClustering, OPTICS, BIRCH

**Attention :** pour certains algorithmes, vous devrez jouer avec les paramètres à votre disposition pour parvenir à obtenir au moins deux clusters.

In [98]:
from sklearn.cluster import DBSCAN, SpectralClustering, AffinityPropagation, OPTICS, Birch

algorithms =[
    KMeans(n_clusters=2, n_init='auto'),
    AgglomerativeClustering(n_clusters=2, linkage='average'),
    DBSCAN(eps=0.1),
    SpectralClustering(n_clusters=2),
    AffinityPropagation(),
    OPTICS(),
    Birch(n_clusters=2, threshold=0.2)
]

silhouette_scores = []
algorithms_names = ['KMeans','AgglomerativeClustering','DBSCAN','SpectralClustering','AffinityPropagation','OPTICS','Birch']


for algo in algorithms:
    y_algo = algo.fit_predict(X_normalized)
    silhouette_scores.append(silhouette_score(X_normalized, y_algo))
  
    
plt.bar(algorithms_names,silhouette_scores)
plt.xticks(rotation=90)
plt.xlabel("Algorithmes")
plt.ylabel("Score de silhouette")
plt.title("Score de silhouette en fonction de l'alogoritmes utiliser")
plt.show()